In [ ]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [ ]:
print(text[:250])

In [ ]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

In [ ]:
seq_length = 100  
examples_per_epoch = len(text)//(seq_length+1)


char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):  
    input_text = chunk[:-1]  
    target_text = chunk[1:]  
    return input_text, target_text  

dataset = sequences.map(split_input_target)  

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))

In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
print(len(example_batch_predictions))
print(example_batch_predictions)

In [ ]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

In [ ]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

In [ ]:
sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars 

"kP?'$Kx3e,TW?kpQNGrUL?zZeEgGDPFB3;gyDuWKt,hOMsOJfKlr?B\nM-G.?m,?Xrcc3yWZc&toINJAAHGjAJjyqqi\nA!vJuRKKC"

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
   
  num_generate = 800

  
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  
  text_generated = []

  
  temperature = 1.0

 
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      
    
      predictions = tf.squeeze(predictions, 0)

      
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeour planers, strew dimple-merry, till his mouth
What thou well darkly and unailed by the gods;
To through all the vessel: what do you know
Of sudden actunches with your blood!
K:
I will not sup to-night.

BENVOLIO:
'Tis pity the portions' stands the mustard.

MERCUTIO:
I will be shriving, and have thou shalt not stay.
I tell thee, es a very sime,
Anoone old him too. This face the west,
An any bed--
LUKIO:
Take hands there died, Prepurer fair?
I saw the help of mileness, beggars
Which so richesome best
With pity, light, one hour from his slaimentake not, then I'll fight it would;
For never was a gin his curreht blame:
If this foull is present at this fair
May year a subject for a neellow?

MENENIUS:

DUKE VINCENTIO:
This overth than in the eye, the honour'd move
souls this branch of pardon:

